In [48]:
import random
from scipy import special as sc
#Constants ,experiment parameters
NUM_QUEENS = 4
POPULATION_SIZE =10
MIXING_NUMBER = 2
MUTATION_RATE = 0.05

In [49]:
def generate_population():
  population = []

  for individual in range(POPULATION_SIZE):
    new = [random.randrange(NUM_QUEENS) for idx in range(NUM_QUEENS)]
    population.append(new)

  return population

In [50]:
#Running the experiment

generation = 0

#generate random population
population = generate_population()

In [51]:
population

[[0, 3, 1, 1],
 [2, 0, 3, 1],
 [0, 0, 0, 1],
 [1, 1, 2, 1],
 [1, 3, 0, 3],
 [3, 3, 0, 1],
 [2, 2, 2, 1],
 [0, 1, 1, 1],
 [2, 3, 0, 0],
 [1, 1, 3, 3]]

In [52]:
def fitness_score(seq):
  score =0

  for row in range(NUM_QUEENS):
    col = seq[row]

    for other_row in range(NUM_QUEENS):

      #queens cannot pair with itself
      if other_row == row:
        continue

      if seq[other_row] == col:
        continue

      if other_row + seq[other_row] == row+col:
        continue

      if other_row - seq[other_row] == row-col:
        continue

      # score ++ if every pair of queens are non-attacking
      score+=1

  #divide by 2 as pairs of queens are commutative
  return score/2

In [53]:
for ind in population:
  print (fitness_score(ind))

4.0
6.0
2.0
1.0
5.0
3.0
2.0
2.0
3.0
2.0


In [54]:
def selection(population):
  parents =[]

  for ind in population:
    #select parents with probability proportional to their fitness score
    if random.randrange(sc.comb(NUM_QUEENS,2)*2) < fitness_score(ind):
      parents.append(ind)

  return parents

In [55]:
from IPython.core.formatters import PlainTextFormatter
import itertools 

def crossover(parents):

  #random indexes to cross states with
  cross_points = random.sample(range(NUM_QUEENS) , MIXING_NUMBER -1)
  offsprings =[]

  #all permutations
  permutations = list(itertools.permutations(parents,MIXING_NUMBER))

  for perm in permutations:
    offspring = []

    #tracking starting index of sublist
    start_pt =0

    for parents_idx, cross_point in enumerate(cross_points): #doesn't account for last permutations
      #sublist of parent to be crossed
      parent_part = perm[parents_idx][start_pt:cross_point]
      offspring.append(parent_part)
      #update index pointer
      start_pt = cross_point
    #last parent
    last_parent = perm[-1]
    parent_part = last_parent[cross_point:]
    offsprings.append(parent_part)


    #flatten the list since append works kinda efficient
    offsprings.append(list(itertools.chain(*offspring)))

  return offsprings

In [56]:
def mutate(seq):
  for row in range(len(seq)):
    if random.random() < MUTATION_RATE:
      seq[row] = random.randrange(NUM_QUEENS)
    
    return seq

In [57]:
def print_found_goal(popualtion,to_print = True):
  for ind in population:
    score = fitness_score(ind)

    if to_print:
      print(f'{ind}.Score:{score}')
    if score == sc.comb(NUM_QUEENS,2):
      if to_print:
        print('Solution found')
      return True

  if to_print:
    print('Solution not found')
  return False

In [58]:
def GA(population):

  #select individuals to become parents
  parents = selection(population)

  #recombination.Create new Offsprings
  offsprings = crossover(parents)

  #mutation
  offsprings = list(map(mutate,offsprings))

  #introduce top-scoring individuals from previous generation and keep top fitness individuals
  new_gen = offsprings

  for ind in population:
    new_gen.append(ind)

  new_gen = sorted(new_gen,key=lambda ind:fitness_score(ind),reverse= True)[:POPULATION_SIZE]

  return new_gen


In [59]:
#Running the experiment

generation = 0

#generate random population 
population = generate_population()

while not print_found_goal(population):
  print(f'Genration:{generation}')
  print_found_goal(population)
  population = GA(population)
  generation+=1

[2, 3, 2, 2].Score:1.0
[2, 1, 0, 0].Score:2.0
[0, 3, 0, 3].Score:3.0
[0, 2, 3, 1].Score:5.0
[2, 0, 0, 0].Score:2.0
[0, 1, 2, 0].Score:2.0
[3, 0, 1, 1].Score:3.0
[2, 0, 1, 3].Score:5.0
[3, 2, 1, 3].Score:2.0
[1, 0, 0, 0].Score:2.0
Solution not found
Genration:0
[2, 3, 2, 2].Score:1.0
[2, 1, 0, 0].Score:2.0
[0, 3, 0, 3].Score:3.0
[0, 2, 3, 1].Score:5.0
[2, 0, 0, 0].Score:2.0
[0, 1, 2, 0].Score:2.0
[3, 0, 1, 1].Score:3.0
[2, 0, 1, 3].Score:5.0
[3, 2, 1, 3].Score:2.0
[1, 0, 0, 0].Score:2.0
Solution not found


TypeError: ignored